In [1]:
# Importació de llibreries per al tractament de dades i aprenentatge automàtic clàssic
import numpy as np
from sklearn.datasets import make_moons
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Importació de components de Qiskit per a la construcció de circuits i càlculs d'observables
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Importació del backend simulat i l’estimador de Qiskit Runtime
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime.fake_provider import FakeAlmadenV2

# ------------------------------------------------------
# 1. Generació i pre-processament del conjunt de dades
# ------------------------------------------------------

# Es genera un conjunt de dades sintètic de dues dimensions no linealment separables
X, y = make_moons(n_samples=20, noise=0.1, random_state=0)

# S’estandarditzen les dades per a garantir una codificació quàntica òptima
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Es divideixen les dades en conjunt d’entrenament i conjunt de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ------------------------------------------------------
# 2. Definició del circuit de codificació (feature map)
# ------------------------------------------------------

# Es defineix una funció que implementa un mapa de característiques quàntic senzill
# mitjançant rotacions Ry i Rz combinades amb una porta CZ per generar entrellaçament
def feature_map(x):
    qc = QuantumCircuit(2)
    qc.ry(x[0], 0)
    qc.rz(x[1], 1)
    qc.cz(0, 1)
    return qc

# ------------------------------------------------------
# 3. Implementació de la funció de càlcul de la fidelitat
# ------------------------------------------------------

# Aquesta funció calcula la fidelitat entre dues mostres codificades quànticament
# utilitzant la primitiva EstimatorV2 sobre un backend simulat amb layout físic realista
def kernel_fidelity_estimator(x1, x2, estimator, pm):
    # Es construeix el circuit combinat U(x1)·U†(x2)
    qc = QuantumCircuit(2)
    qc.append(feature_map(x1), [0, 1])
    qc.append(feature_map(x2).inverse(), [0, 1])

    # Es defineix un observable compost per les combinacions de Pauli que permet mesurar la fidelitat
    observable = SparsePauliOp.from_list([
        ("II", 0.25),
        ("IZ", 0.25),
        ("ZI", 0.25),
        ("ZZ", 0.25)
    ])

    # El circuit es transpila per adaptar-se al layout físic del backend simulat
    isa_circuit = pm.run(qc)
    mapped_observable = observable.apply_layout(isa_circuit.layout)

    # L’estimador s’utilitza per obtenir el valor esperat (fidelitat) del circuit
    job = estimator.run([(isa_circuit, mapped_observable)])
    pub_result = job.result()[0]
    fidelity = pub_result.data.evs

    return fidelity.real

# ------------------------------------------------------
# 4. Construcció de la matriu de kernel quàntic
# ------------------------------------------------------

# Aquesta funció construeix la matriu simètrica de fidelitats entre parelles de mostres
# que posteriorment s’utilitzarà com a kernel per a entrenar el model clàssic
def compute_kernel_matrix(X1, X2, estimator, pm):
    n1, n2 = len(X1), len(X2)
    kernel_matrix = np.zeros((n1, n2))
    for i in range(n1):
        for j in range(n2):
            fidelity = kernel_fidelity_estimator(X1[i], X2[j], estimator, pm)
            kernel_matrix[i, j] = fidelity
            print(f"Fidelity ({i}, {j}): {fidelity:.4f}")
    return kernel_matrix

# ------------------------------------------------------
# 5. Inicialització del backend i entorn de simulació
# ------------------------------------------------------
backend = FakeAlmadenV2()
print(f"Using backend: {backend.name}")
estimator = Estimator(backend)
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

# ------------------------------------------------------
# 6. Entrenament i avaluació del model amb SVM
# ------------------------------------------------------

# Es calcula la matriu de kernel quàntic utilitzant la fidelitat entre estats codificats
print("Computing kernel matrix using FakeAlmadenV2 simulator...")
K_train = compute_kernel_matrix(X_train, X_train, estimator, pm)
K_test = compute_kernel_matrix(X_test, X_train, estimator, pm)

# Es construeix un model SVM amb kernel precomputat (la matriu de fidelitat quàntica)
svc = SVC(kernel='precomputed')
svc.fit(K_train, y_train)

# Es realitza la predicció i es calcula l’accuràcia sobre el conjunt de test
y_pred = svc.predict(K_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {acc * 100:.2f}%")


Using backend: fake_almaden
Computing kernel matrix using FakeAlmadenV2 simulator...
Fidelity (0, 0): 0.9788
Fidelity (0, 1): 0.5386
Fidelity (0, 2): 0.8860
Fidelity (0, 3): 0.7153
Fidelity (0, 4): 0.7720
Fidelity (0, 5): 0.9766
Fidelity (0, 6): 0.7859
Fidelity (0, 7): 0.2627
Fidelity (0, 8): 0.8259
Fidelity (0, 9): 0.9736
Fidelity (0, 10): 0.9746
Fidelity (0, 11): 0.5984
Fidelity (0, 12): 0.8926
Fidelity (0, 13): 0.9404
Fidelity (0, 14): 0.5586
Fidelity (0, 15): 0.8823
Fidelity (1, 0): 0.5269
Fidelity (1, 1): 0.9829
Fidelity (1, 2): 0.8105
Fidelity (1, 3): 0.0977
Fidelity (1, 4): 0.9099
Fidelity (1, 5): 0.6213
Fidelity (1, 6): 0.1516
Fidelity (1, 7): 0.8896
Fidelity (1, 8): 0.8701
Fidelity (1, 9): 0.4607
Fidelity (1, 10): 0.4387
Fidelity (1, 11): 0.0654
Fidelity (1, 12): 0.8022
Fidelity (1, 13): 0.3545
Fidelity (1, 14): 0.9819
Fidelity (1, 15): 0.2273
Fidelity (2, 0): 0.8860
Fidelity (2, 1): 0.8101
Fidelity (2, 2): 0.9810
Fidelity (2, 3): 0.4199
Fidelity (2, 4): 0.9485
Fidelity (2, 5)